In [3]:
pip install xmltodict

In [4]:
import requests 
import xmltodict 
import json 
import datetime
from urllib import parse
import pandas as pd


In [51]:
def getCovidKR(end_day, start_day): 
  url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson' #ServiceKey는 url decode 한 값임. 
  serviceKey='wMFqCOL2ZotVqyvmw311lh7pNf%2BKvsIH%2FqiMtduIe%2FYd%2B%2B537ffxw%2Brmt5tfsFMY3Xg%2BGWEWmmAm97QnJM6RMw%3D%3D'
  serviceKey_decode=parse.unquote(serviceKey)

  payload = {'ServiceKey':serviceKey_decode, 'startCreateDt':start_day, 'endCreateDt':end_day } 
  
  res = requests.get(url, params=payload) 
  if (res.status_code == 200): # Ordered dictionary type
    result = xmltodict.parse(res.text) #dictionlay type 
    dd = json.loads(json.dumps(result)) 
    #print(dd)
    data = pd.json_normalize(dd['response']['body']['items']['item']) ## json_normalize를 이용해서 계층 구조를 없애고 dataframe으로 불러옴 
    #print(data)
    covid_data = data[['stateDt','decideCnt']]
    covid_data['stateDt'] = pd.to_datetime(covid_data['stateDt'], format='%Y-%m-%d')
    #covid_data.astype({'decideCnt':'int'}) ## 타입변환 안됨
    #pd.to_numeric(covid_data['decideCnt']) ## 타입변환 안됨
    covid_data['decideCnt'] = covid_data['decideCnt'].apply(pd.to_numeric)
    covid_data.drop_duplicates("stateDt", keep="first", inplace=True) ## 중복값 제거
    covid_data.sort_values(by=['stateDt'], inplace=True, ascending=False)  ## 날짜순 정렬
    covid_data['decideCnt'] = (covid_data.decideCnt - covid_data.decideCnt.shift(-1)).fillna(0)
    print(covid_data)
    
  else: 
    print ('res.status_code is NOT ok')



In [55]:
if __name__ == "__main__": 
  today =datetime.datetime.now() 
  yesterday = today - datetime.timedelta(1) 
  d1 = today.strftime("%Y%m%d")
  d2 = '20200301'
  getCovidKR(d1,d2)



       stateDt  decideCnt
0   2020-12-30     1050.0
1   2020-12-29     1044.0
2   2020-12-28      807.0
3   2020-12-27      970.0
4   2020-12-26     1132.0
..         ...        ...
305 2020-03-05      438.0
306 2020-03-04      516.0
307 2020-03-03      600.0
308 2020-03-02      476.0
309 2020-03-01        0.0

[305 rows x 2 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r